In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import numpy as np
import string
from string import ascii_lowercase
import multiway_decision_tree as mdt
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import LinearSVC
import pickle

In [3]:
class Split_actor:
    def __init__( self, attr ):
        self.attr = attr
        
    def get_attr( self ):
        return self.attr
    
    def split( self, tst_pt, ancestors ):
        if len( ancestors ) == 0:
            mask = input( "Start  : " )
        else:
            mask = input( "Guess " + str( self.attr ) + ": " )
        
        # Splitting a single point so the dictionary is trivial
        split_dict = {}
        split_dict[ mask ] = ( [ 0 ], [ mask ] )
        
        return split_dict
        
    def default_predict( self, tst_pt, ancestors ):
        return [ None ]
    
class Leaf_actor:
    def __init__( self, data ):
        self.data = data
        
    def predict( self, tst_pt, ancestors ):
        return [ self.data[0] ]
        
    def default_predict( self, tst_pt, ancestors ):
        return [ None ]

def get_size( trn_pts ):
    return len( trn_pts )

def is_pure_enough( trn_pts ):
    if len( trn_pts ) < 2:
        return True
    else:
        return False

def get_leaf_actor( trn_pts, ancestors ):
    return Leaf_actor( data = trn_pts )

# See the following StackOverflow conversation on some alternate ways to compute entropy
# https://stackoverflow.com/questions/15450192/fastest-way-to-compute-entropy-in-python
def get_entropy( counts ):    
    # Why include elements that do not appear at all?
    assert np.min( counts ) > 0, "Elements with zero or negative counts detected"
    
    num_elements = counts.sum()
    
    # A singular or empty set has zero entropy
    if num_elements <= 1:
        print( f"warning: { num_elements } elements in total." )
        return 0
    
    proportions = counts / num_elements
    
    return np.sum( proportions * np.log2( counts ) )  

def get_mask( char_list, word ):
    mask = []
    
    for c in word:
        if c in char_list:
            mask.append( c )
        else:
            mask.append( '_' )
    
    return ' '.join( mask )

def try_attr( char_list, words ):
    split_dict = {}
    count_dict = {}
    
    for word in words:
        mask = get_mask( char_list, word )
        
        if mask not in split_dict:
            split_dict[ mask ] = []
            count_dict[ mask ] = 0
            
        split_dict[ mask ].append( word )
        count_dict[ mask ] += 1

    entropy = get_entropy( np.array( list( count_dict.values() ) ) )
    
    return ( entropy, split_dict )

def get_split_actor( trn_pts, ancestors ):
    
    # If this is the root node, we must split on word length
    if len( ancestors ) == 0:
        ( entropy, split_dict ) = try_attr( [], trn_pts )
        return ( Split_actor( "len" ), split_dict )
        
    # Exclude the first element of the ancestry since that is the length split
    ancestor_attr = [ split[0] for split in ancestors ][1:]
    
    best_entropy = np.inf
    best_attr = ''
    best_split_dict = None
    
    for c in [ char for char in ascii_lowercase if char not in ancestor_attr ]:
        char_list = ancestor_attr.copy()
        char_list.append( c )
        ( entropy, split_dict ) = try_attr( char_list, trn_pts )
#         print( f"{c}({entropy})", end = ", " )
        if entropy < best_entropy:
            best_entropy = entropy
            best_attr = c
            best_split_dict = split_dict
    
#     print()
#     print( f"-->{best_attr}({best_entropy})\n" )
    
    return ( Split_actor( best_attr ), best_split_dict )

In [4]:
f = open( "base", 'r' )
words = f.read().split( '\n' )[:-1]        # Omit the last line since it is empty
f.close()

In [5]:
max_depth = 10
dtID3 = mdt.Tree( min_leaf_size = 1, max_depth = max_depth )
dtID3.train( words, get_split_actor, get_leaf_actor, is_pure_enough, get_size, verbose = True )

with open( "dt.mdl", "wb" ) as f:
    pickle.dump( dtID3, f )

root
└───len
    ├───e
    │   ├───a
    │   │   ├───g
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───o
    │   │   │   ├───g
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───c
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───i
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───c
    │   │   │       ├───█
    │   │   │       └───█
    │   │   ├───█
    │   │   ├───█
    │   │   ├───c
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───n
    │   │   │   ├───l
    │   │   │   │   ├───h
    │   │   │   │   │   ├───k
    │   │   │   │   │   │   ├───b
    │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   └───█
    │   │   │   │

    │   │   │   │   ├───n
    │   │   │   │   │   ├───a
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───i
    │   │   │   │   ├───a
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───h
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───a
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   ├───n
    │   │   │   │   ├───a
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │ 

    │   │   │   │   │   │   │   ├───c
    │   │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   │   ├───d
    │   │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   │   ├───f
    │   │   │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   ├───c
    │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   └───m
    │   │   │   │   │   │   │       ├───█
    │   │   │   │   │   │   │       └───█
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   ├───b
    │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   ├───s
    │   │   │   │   │   │   ├───c
    │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   └─

    │   ├───s
    │   │   ├───a
    │   │   │   ├───c
    │   │   │   │   ├───█
    │   │   │   │   └───l
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   └───d
    │   │   │       ├───█
    │   │   │       └───n
    │   │   │           ├───█
    │   │   │           └───█
    │   │   ├───n
    │   │   │   ├───c
    │   │   │   │   ├───f
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───█
    │   │   │   ├───r
    │   │   │   │   ├───v
    │   │   │   │   │   ├───a
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   └───l
    │   │   │   │   │       ├───█
    │   │   │   │   │       └───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   └───█
    │   │   ├───i
    │   │   │   ├───█
    │   │   │   └───█
    │   │   └───█
    │   ├───r
    │   │   ├───f
    │   │   │   ├───d
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   └───v
    │   │   │       ├─

    │   │   │   │   │   │           └───f
    │   │   │   │   │   │               ├───█
    │   │   │   │   │   │               └───█
    │   │   │   │   │   └───█
    │   │   │   │   └───o
    │   │   │   │       ├───i
    │   │   │   │       │   ├───a
    │   │   │   │       │   │   ├───c
    │   │   │   │       │   │   │   ├───█
    │   │   │   │       │   │   │   └───█
    │   │   │   │       │   │   ├───n
    │   │   │   │       │   │   │   ├───█
    │   │   │   │       │   │   │   ├───█
    │   │   │   │       │   │   │   └───t
    │   │   │   │       │   │   │       ├───█
    │   │   │   │       │   │   │       └───█
    │   │   │   │       │   │   └───h
    │   │   │   │       │   │       ├───█
    │   │   │   │       │   │       ├───█
    │   │   │   │       │   │       └───█
    │   │   │   │       │   ├───p
    │   │   │   │       │   │   ├───█
    │   │   │   │       │   │   ├───k
    │   │   │   │       │   │   │   ├───█
    │   │   │   │       │   │   │   └───m
    │   │ 

    │   │   ├───a
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───m
    │   │   │   │   ├───█
    │   │   │   │   ├───f
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───n
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───l
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───h
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───l
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   ├───c
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
  

    │   │   ├───o
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───a
    │   │   │       ├───█
    │   │   │       └───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───t
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───c
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───c
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█


    │   │   │   │   ├───l
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───n
    │   │   │   │   ├───d
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   └───l
    │   │   │   │       ├───█
    │   │   │   │       └───m
    │   │   │   │           ├───█
    │   │   │   │           └───█
    │   │   │   ├───l
    │   │   │   │   ├───m
    │   │   │   │   │   ├───█
    │   │   │   │   │   ├───d
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───s
    │   │   │   │   │   ├───c
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───n
    │   │   │   │   ├───█
    │   │   │   │   ├───d
    │   │   │   │   │   ├───█
  

    │   ├───r
    │   │   ├───l
    │   │   │   ├───n
    │   │   │   │   ├───d
    │   │   │   │   │   ├───h
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   ├───c
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───b
    │   │   │   │   │   ├───h
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   └───a
    │   │   │   │   │       ├───█
    │   │   │   │   │       └───█
    │   │   │   │   ├───c
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───a
    │   │   │   │   │   ├───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───p
    │   │   │   │   │       ├───█
    │   │   │   │   │       └───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   

    │   ├───a
    │   │   ├───█
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───█
    │   ├───r
    │   │   ├───█
    │   │   ├───█
    │   │   ├───█
    │   │   └───█
    │   ├───a
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───g
    │   │   ├───█
    │   │   ├───█
    │   │   └───█
    │   ├───a
    │   │   ├───█
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───a
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───█
    │   └───█
    ├───n
    │   ├───c
    │   │   ├───█
    │   │   ├───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───█
    │   │   ├───s
    │   │   │   ├───█
    │   │   │   └───█
    │   │   └───█
    │   ├───a
    │   │   ├───█
    │   │   └───█
    │   ├───a
    │   │   ├───█
    │   │   ├───█
    │   │   └───█
    │   ├───a
    │   │   ├───█
    │   │   └───█
    │   ├───i
    │   

In [7]:
dtID3.predict( [ [] ] )

Start  : _ _ _ _ _ _ _ _
Guess e: _ _ _ _ _ _ _ _
Guess i: _ i _ _ _ _ _ _
Guess r: _ i r _ _ _ _ _


['birthday']